Importing libraries

In [1]:
#introducing libraries
!pip install numpy scipy pandas>1.0 matplotlib lmfit colour
#calling up libraries
from datetime import datetime
import pandas as pd
import numpy as np
import scipy
from scipy import optimize
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.stats import linregress
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from mpl_toolkits import mplot3d
from datetime import timedelta
from lmfit.models import LorentzianModel
import statistics
from matplotlib.collections import LineCollection 
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from scipy.signal import argrelmax
#import CSV data from drive
from google.colab import drive
import glob
import os
drive.mount('drive', force_remount=True)
folder = "/content/drive/MyDrive/P3HT-PCBM TA Group import"
all_files = glob.glob(os.path.join(folder, '*.csv'))
Spec_data = {}
Sorb_data = {}
Irr_data = {}
titles = []



for filename in all_files:
  #Simplify name
  new_name = filename.replace(".csv", "").replace("/content/drive/MyDrive/P3HT-PCBM TA Group import/","")
  #import data
  df = pd.read_csv(filename, sep=',', header=0)

  #Some cleaning
  df = df.rename({'0.00000E+0.1':'0.000000000'}, axis=1)
  cols = df.columns.values
  cols[0] = 'Wavelength(nm)'
  df.columns = cols
  #remove extraneous data at the end
  df = df[pd.to_numeric(df['Wavelength(nm)'], errors='coerce').notnull()]
  df['Wavelength(nm)'] = df['Wavelength(nm)'].astype(float)
  df = df.set_index('Wavelength(nm)')
  #remove rows (wavelengths) with at least 3 na values
  df = df.dropna(axis=0, thresh=3)
  df = df.dropna(axis=1, thresh=3)

  #change strings to numbers
  df = df.astype(float)
  df.columns = pd.to_numeric(df.columns)

  titles.append(new_name)
  Spec_data[new_name] = df

Spec_data



Mounted at drive


{'P3HT neat film bhj .7mW ChlBnz 20min run1 UV-Vis (corrected)':                 -4.93249     -4.43249     ...   6623.22000   6999.82000
 Wavelength(nm)                            ...                          
 335.695                 NaN    -0.000047  ...    -0.000207     0.000030
 336.268                 NaN     0.000139  ...    -0.000088    -0.000219
 336.841                 NaN     0.000127  ...    -0.000004    -0.000005
 337.413                 NaN     0.000151  ...     0.000328    -0.000246
 337.986                 NaN     0.000041  ...     0.000301    -0.000259
 ...                     ...          ...  ...          ...          ...
 825.485           -0.000305     0.000135  ...     0.001131     0.001159
 826.090            0.000897     0.000430  ...     0.001883     0.000331
 826.695           -0.000002    -0.000024  ...    -0.000385     0.000748
 827.300            0.000568    -0.000566  ...    -0.000861     0.000107
 827.905            0.000614     0.001061  ...     0.000560 

In [2]:
folder = "/content/drive/MyDrive/Data from Lit"
all_files = glob.glob(os.path.join(folder, '*.csv'))
Lit_data = {}



for filename in all_files:
  #Simplify name
  new_name = filename.replace(".csv", "").replace("/content/drive/MyDrive/Data from Lit/","")
  #import data
  df = pd.read_csv(filename, sep=',', header=None)
  df.columns = ['Wavelength(nm)','ΔA']
  #remove extraneous data at the end
  df = df[pd.to_numeric(df['Wavelength(nm)'], errors='coerce').notnull()]
  df['Wavelength(nm)'] = df['Wavelength(nm)'].astype(float)
  df = df.set_index('Wavelength(nm)')
  df = df.sort_index()
  print(new_name)
  print(df)
  Lit_data[new_name] = df

1000 Clafton2013 rr-P3HT NP sol TA 1000ps
                      ΔA
Wavelength(nm)          
853.802356      0.004112
870.826164      0.004125
887.841134      0.004083
904.865139      0.004097
921.881126      0.004062
938.903982      0.004068
955.925841      0.004069
972.947831      0.004071
989.965616      0.004046
1006.990184     0.004064
1024.011307     0.004060
1041.031650     0.004051
1058.053877     0.004054
1075.073938     0.004044
1092.097553     0.004055
1109.115534     0.004032
1126.138325     0.004038
1143.159946     0.004038
1160.181784     0.004038
1177.202191     0.004030
1194.738755     0.004024
1211.241576     0.004004
1228.262244     0.003998
1245.285642     0.004008
1262.309690     0.004022
1279.327367     0.003997
1296.349335     0.003998
1313.374163     0.004018
1330.396044     0.004018
1347.422649     0.004048
1356.706697     0.004045
1000 Clafton2013 P3HT-PCBM NP sol TA 1000ps
                          ΔA
Wavelength(nm)              
854.218155      2.974728e-04
87

In [3]:
#routine for switching between type comparisons
def switch(name):
  if ('NPs' in name) and ('film' in name):
    return 'Blend Nanoparticles in Annealed Film'
  elif ('sol' in name) and ('NPs' in name):
    return 'Blend Nanoparticles in Solution'
  elif ('bhj' in name) and ('film' in name):
    k = name.split(sep=' ')[0]
    return k + ' in Bulk Annealed Film'
  else:
    return ' '

f=10000
#for excitations, hc needs to be in uJ*nm
hc = 1.98644586*(10**(-10))
pump_wl = 400
###Note to self: ADD IN OD data###
OD = 0.4
beam_area = 3.1415926536*(0.01**2)

def fluence_translate(name):
  #note units are uJ/cm^2
  P = 0
  words = name.split()
  for word in words:
    if ('uW' in word):
      P = float(word.replace('uW', ''))
    if ('mW' in word):
      P = 1000*float(word.replace('mW',''))
  photon_E = hc/pump_wl
  photons_absorbed = 1 - (10**(-OD))
  #Excitation Density
  #return photons_absorbed*P*0.8/(f*photon_E*beam_area)
  #E per pulse per area
  return P/(f*beam_area)




for name in Spec_data:
  print(fluence_translate(name))


222.81692032792952
0.0
0.0
7.639437268386155
31.830988618275647
146.74085753025074
311.94368845910134
3119.4368845910135
6.366197723655129
31.830988618275647
159.15494309137824
311.94368845910134
3087.6058959727375
311.94368845910134
3119.4368845910135


Importing data from google drive folder, sorting each into absorbance, spectroscope runs, and general spectroscope data.

In [ ]:
def spec_time_data(data, name, times):
  for i in range(len(times)):
      x = np.asarray(data[name].index)
      y = np.asarray(data[name][times[i]])
      if 'UV-Vis' in name:
        y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
        plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
        plt.axvline(pump_wl-10, color='black')
        plt.axvline(pump_wl+10, color='black')
        plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
      plt.plot(x, y,  label=str(dex[i])+' ps', color=colormap(i/len(times)))
      plt.legend(title='Time from t0', bbox_to_anchor=(1,1), loc="upper left")
      plt.title(name + " Transient Spectroscopy")
      plt.xlabel('Wavelength (nm)')
      plt.ylabel('ΔA')
#color method from stack exchange :https://stackoverflow.com/questions/25668828/how-to-create-colour-gradient-in-python
#def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
#    c1=np.array(mpl.colors.to_rgb(c1))
#    c2=np.array(mpl.colors.to_rgb(c2))
#    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
#c1='white' 
#c2='black' 
#better color method
colormap = cm.get_cmap('viridis')
#find global maxes to fix TA scale

bmaxes=[]
bmins=[]

for name in Spec_data:
  bmaxes.append(Spec_data[name].max().max())
  bmins.append(Spec_data[name].min().min())

bmax = max(bmaxes)
bmin = min(bmins)

#https://codereview.stackexchange.com/questions/189319/nearest-neighbor-algorithm-general-neares
def find_nearest(array, number, direction): 
    idx = -1
    if direction is None:
        idx = (np.abs(array - number)).min()
    elif direction == 'backward':
        _delta = number - array
        _delta_positive = _delta[_delta > 0]
        if not _delta_positive.empty:
            idx = _delta_positive.min()
    elif direction == 'forward':
        _delta = array - number
        _delta_positive = _delta[_delta >= 0]
        if not _delta_positive.empty:
            idx = _delta_positive.min()
    return idx

#filling 'times' list with closest values found in time stamps (columns)
for name in Spec_data:
  dex = [1, 10, 100, 1000, 6500]
  times = []
  for i in range(len(dex)):
    times.append(find_nearest(Spec_data[name].columns,dex[i], 'forward')+dex[i])
  spec_time_data(Spec_data, name, times)
  plt.plot(np.array(Spec_data[name].index), np.zeros(len(Spec_data[name].index), dtype=float, order='C'), color='black')
  plt.show()


  print(name)
  plt.show()




Try adding again?

In [ ]:
###Okay, don't worry about the other stuff right now, just print your data out

#BHJ TA

viridis = cm.get_cmap('Greys')
norm=1
t=0
for name in sorted(Lit_data):

  if ('Guo' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    if int(t) == 0:
      norm = abs(y).max()
    print(t, norm)
    plt.plot(x, y/norm, label=t + 'ps (Guo et al. 2011)', linestyle='--',color=viridis(np.log(int(t)+2)/np.log(5002)))




df = Spec_data['P3HT-PCBM bhj film 700uW min run2 UV-Vis (corrected)']
norm = abs(df[find_nearest(df.columns, 1, 'forward')]).max()
t0 = abs(df).max().idxmax(axis=1)

d1 = [1,10, 100, 3000]
dex = d1
times = []


for i in range(len(dex)):
  times.append(find_nearest(df.columns,dex[i], 'forward')+dex[i])
for i in range(len(times)):
  x = np.asarray(df.index)
  y = np.asarray(df[times[i]].rolling(3).mean())/norm
  y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
  plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
  plt.axvline(pump_wl-10, color='black')
  plt.axvline(pump_wl+10, color='black')
  plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
  plt.plot(x, y,  label=str(int(d1[i]))+' ps', color=viridis(np.log(int(d1[i])+2)/np.log(5002)))
plt.plot(np.array(df.index), np.zeros(len(df.index), dtype=float, order='C'), color='black')
plt.title("P3HT-PCBM BHJ film Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA (normalized)')
plt.xlim(380, 1350)
plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(1.55,0.5))
plt.show()


In [ ]:
#JUST P3HT

viridis = cm.get_cmap('bone')
norm=1
t=0
for name in sorted(Lit_data):

  if ('Guo' in name) and ('neat' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    if int(t) == 0:
      norm = abs(y[x<800]).max()
    print(t, norm)
    plt.plot(x, y/norm, label=t + 'ps (Guo et al. 2011)', linestyle='--',color=viridis(np.log(int(t)+2)/np.log(5002)))




df = Spec_data['P3HT neat film bhj .7mW ChlBnz 20min run1 UV-Vis (corrected)']
df = df[df.index<800]
df = df[df.index>405]
norm = abs(df[find_nearest(df.columns, 1, 'forward')][df.index>550]).max()
t0 = abs(df).max().idxmax(axis=1)

d1 = [1,10, 100, 5000]
dex = d1
times = []


for i in range(len(dex)):
  times.append(find_nearest(df.columns,dex[i], 'forward')+dex[i])
for i in range(len(times)):
  x = np.asarray(df.index)
  y = np.asarray(df[times[i]].rolling(3).mean())/norm
  y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
  plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
  plt.axvline(pump_wl-10, color='black')
  plt.axvline(pump_wl+10, color='black')
  plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
  plt.plot(x, y,  label=str(int(d1[i]))+' ps', color=viridis(np.log(int(d1[i])+2)/np.log(10002)))
plt.plot(np.array(df.index), np.zeros(len(df.index), dtype=float, order='C'), color='black')
plt.title("P3HT neat film Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA (normalized)')
plt.xlim(380, 800)
plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(1.55,0.5))
plt.show()

In [ ]:
#NP film TA
viridis = cm.get_cmap('YlOrRd')
df = Spec_data['P3HT-PCBM NPs film 500uW UV-Vis (corrected)']
df = df[df.index<800]
df = df[df.index>405]
norm = abs(df[find_nearest(df.columns, 1, 'forward')]).max()
t0 = abs(df).max().idxmax(axis=1)

d1 = [1,10, 100, 1000, 5000]
dex = d1 
times = []


for i in range(len(dex)):
  times.append(find_nearest(df.columns,dex[i], 'forward')+dex[i])
for i in range(len(times)):
  x = np.asarray(df.index)
  y = np.asarray(df[times[i]].rolling(3).mean())/norm
  y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
  plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
  plt.axvline(pump_wl-10, color='black')
  plt.axvline(pump_wl+10, color='black')
  plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
  plt.plot(x, y,  label=str(int(d1[i]))+' ps', color=viridis(np.log(int(d1[i])+2)/np.log(5002)))
times=[]

df_NIR = Spec_data['P3HT-PCBM NPs agg film 980uW NIR (corrected)']
df_NIR = df_NIR[df_NIR.index > 850]
df_NIR = df_NIR[df_NIR.index < 1350]
for i in range(len(dex)):
  times.append(find_nearest(df_NIR.columns,dex[i], 'forward')+dex[i])
for i in range(len(times)):
  x = np.asarray(df_NIR.index)
  y = np.asarray(df_NIR[times[i]].rolling(3).mean())/(0.5*norm)

  plt.axvspan(800, 850, color='lightgray')
  plt.axvline(800, color='black')
  plt.axvline(850, color='black')
  plt.text(810,0, 'NOISE',rotation='vertical', va='center')
  plt.plot(x, y, color=viridis(np.log(int(d1[i])+2)/np.log(5002)))

plt.plot(np.array(df.index), np.zeros(len(df.index), dtype=float, order='C'), color='black')
plt.plot(np.array(df_NIR.index), np.zeros(len(df_NIR.index), dtype=float, order='C'), color='black')

plt.title("P3HT-PCBM NP Film Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA (normalized)')

plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(0.8,0.5))
plt.show()

In [ ]:
## MY NP SOLUTION

df = Spec_data['P3HT-PCBM NPs solution 461uW 2mm  UV-Vis (corrected)']
df = df[df.index<800]
df=df[df.index>405]
norm = abs(df[find_nearest(df.columns, 1, 'forward')]).max()
t0 = abs(df).max().idxmax(axis=1)

d1 = [0, 1,10, 100, 1000, 5000]
dex = d1+t0 
times = []


for i in range(len(dex)):
  times.append(find_nearest(df.columns,dex[i], 'forward')+dex[i])
for i in range(len(times)):
  x = np.asarray(df.index)
  y = np.asarray(df[times[i]].rolling(3).mean())/norm
  y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
  plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
  plt.axvline(pump_wl-10, color='black')
  plt.axvline(pump_wl+10, color='black')
  plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
  plt.plot(x, y,  label=str(int(d1[i]))+' ps', color=viridis(np.log(int(d1[i])+2)/np.log(5002)))
times=[]
for i in range(len(dex)):
  times.append(find_nearest(Spec_data['P3HT-PCBM NPs sol 980uW 2mm NIR (corrected)'].columns,dex[i], 'forward')+dex[i])
df_NIR = Spec_data['P3HT-PCBM NPs sol 980uW 2mm NIR (corrected)']
df_NIR = df_NIR[df_NIR.index > 850]
df_NIR = df_NIR[df_NIR.index < 1350]

for i in range(len(times)):
  x = np.asarray(df_NIR.index)
  y = np.asarray(df_NIR[times[i]])*1.8/norm

  plt.axvspan(800, 850, color='lightgray')
  plt.axvline(800, color='black')
  plt.axvline(850, color='black')
  plt.text(810,0, 'NOISE',rotation='vertical', va='center')
  plt.plot(x, y, color=viridis(np.log(int(d1[i])+2)/np.log(5002)))

plt.plot(np.array(df.index), np.zeros(len(df.index), dtype=float, order='C'), color='black')
plt.plot(np.array(df_NIR.index), np.zeros(len(df_NIR.index), dtype=float, order='C'), color='black')

plt.title("P3HT-PCBM NP Solution Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA (normalized)')

plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(0.8,0.5))
plt.show()

In [ ]:
# w/REF NP SOLUTION

dex = [1, 10, 100, 1000, 6000]
times = []
for i in range(len(dex)):
  times.append(find_nearest(Spec_data['P3HT-PCBM NPs solution 461uW 2mm  UV-Vis (corrected)'].columns,dex[i], 'forward')+dex[i])
df = Spec_data['P3HT-PCBM NPs solution 461uW 2mm  UV-Vis (corrected)']
df = df[df.index<800]
norm = abs(df[find_nearest(df.columns, 1, 'forward')]).max()

for i in range(len(times)):
  x = np.asarray(df.index)
  y = np.asarray(df[times[i]])
  y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
  plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
  plt.axvline(pump_wl-10, color='black')
  plt.axvline(pump_wl+10, color='black')
  plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
  plt.plot(x, y,  label=str(int(dex[i]-0.6))+' ps', color=viridis((i+1)/(len(times)+1)))
times=[]
for i in range(len(dex)):
  times.append(find_nearest(Spec_data['P3HT-PCBM NPs sol 980uW 2mm NIR (corrected)'].columns,dex[i], 'forward')+dex[i])
df_NIR = Spec_data['P3HT-PCBM NPs sol 980uW 2mm NIR (corrected)']
df_NIR = df_NIR[df_NIR.index > 850]
df_NIR = df_NIR[df_NIR.index < 1350]

for i in range(len(times)):
  x = np.asarray(df_NIR.index)
  y = np.asarray(df_NIR[times[i]])*1.8

  plt.axvspan(800, 850, color='lightgray')
  plt.axvline(800, color='black')
  plt.axvline(850, color='black')
  plt.text(810,0, 'NOISE',rotation='vertical', va='center')
  plt.plot(x, y, color=viridis((i+1)/(len(times)+1)))

plt.plot(np.array(df.index), np.zeros(len(df.index), dtype=float, order='C'), color='black')
plt.plot(np.array(df_NIR.index), np.zeros(len(df_NIR.index), dtype=float, order='C'), color='black')

i=0
for name in sorted(Lit_data):
  maxes_to_normalize = []

  if ('Schwarz' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    maxes_to_normalize.append(y.max())
    maxes_to_normalize.append(abs(y.min()))
    plt.plot(x, y, label=t + 'ps (Schwarz et al. 2015)', linestyle='--', color=viridis(np.log(np.int(t)+1)/np.log(6000)))
    i += 1

for name in sorted(Lit_data):
  maxes_to_normalize = []

  if ('Clafton' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    maxes_to_normalize.append(y.max())
    maxes_to_normalize.append(abs(y.min()))
    plt.plot(x, y, label=t + 'ps (Clafton et al. 2013)', linestyle='--', color=viridis(np.log(np.int(t)+1)/np.log(6000)))
    i += 1


plt.title("P3HT-PCBM NP Solution Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA')

plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(0.8,0.5))
plt.show()

In [ ]:
#JUST LIT FOR MUTUAL COMPARISON

i=0
for name in sorted(Lit_data):
  maxes_to_normalize = []

  if ('Schwarz' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    maxes_to_normalize.append(y.max())
    maxes_to_normalize.append(abs(y.min()))
    plt.plot(x, y, label=t + 'ps (Schwarz et al. 2015)', linestyle='-', color=viridis(np.log(np.int(t)+1)/np.log(6000)))
    i += 1

for name in sorted(Lit_data):
  maxes_to_normalize = []

  if ('Clafton' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    maxes_to_normalize.append(y.max())
    maxes_to_normalize.append(abs(y.min()))
    plt.plot(x, y, label=t + 'ps (Clafton et al. 2013)', linestyle='--', color=viridis(np.log(np.int(t)+1)/np.log(6000)))
    i += 1


plt.title("P3HT-PCBM NP Solution Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA')
plt.xlim(700,1000)
plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(0.8,0.5))
plt.show()

#THEY'LL NEVER COMPARE

In [ ]:
# w/REF NP SOLUTION UV_VIS ONLY
viridis = cm.get_cmap('GnBu')

df = Spec_data['P3HT-PCBM NPs solution 461uW 2mm  UV-Vis (corrected)']
df = df[df.index<800]
df = df[df.index>400]
norm = abs(df).max().max()
t0 = abs(df).max().idxmax(axis=1)

d1 = [0, 1,10, 100, 1000, 6000]
dex = d1+t0 
times = []
for i in range(len(dex)):
  times.append(find_nearest(Spec_data['P3HT-PCBM NPs solution 461uW 2mm  UV-Vis (corrected)'].columns,dex[i], 'forward')+dex[i])
for i in range(len(times)):
  x = np.asarray(df.index)
  y = np.asarray(df[times[i]].rolling(3).mean())/norm
  y[(x>pump_wl-10) & (x<pump_wl+10)] =np.nan
  plt.axvspan(pump_wl-10, pump_wl+10, color='lightgray')
  plt.axvline(pump_wl-10, color='black')
  plt.axvline(pump_wl+10, color='black')
  plt.text(pump_wl-7,0, 'LASER',rotation='vertical', va='center')
  plt.plot(x, y,  label=str(int(d1[i]))+' ps', color=viridis(np.log(int(d1[i])+2)/np.log(5002)))


plt.plot(np.array(df.index), np.zeros(len(df.index), dtype=float, order='C'), color='black')

norm = 1
i=0
for name in sorted(Lit_data):
  maxes_to_normalize = []

  if ('Schwarz' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    data = data[data.index<800]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    if int(t) == 1:
      norm = abs(y).max()
    plt.plot(x, y/norm, label=t + 'ps (Schwarz et al. 2015)', linestyle='--', color=viridis(np.log(np.int(t)+1)/np.log(5002)))
    i += 1



plt.title("P3HT-PCBM NP Solution Transient Spectroscopy UV-Vis")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA (normalized)')

plt.legend(bbox_to_anchor=(1.6,0.5))
plt.show()

In [ ]:
#w/REF NIR COMPARISON NP SOLS
df_NIR = Spec_data['P3HT-PCBM NPs sol 980uW 2mm NIR (corrected)']
df_NIR = df_NIR[df_NIR.index > 850]
df_NIR = df_NIR[df_NIR.index < 1350]

norm = abs(df_NIR).max().max()
t0 = abs(df_NIR).max().idxmax(axis=1)

d1 = [0, 1,10, 100, 1000, 5000]
dex = d1+t0-1 
times = []
for i in range(len(dex)):
  times.append(find_nearest(Spec_data['P3HT-PCBM NPs sol 980uW 2mm NIR (corrected)'].columns,dex[i], 'forward')+dex[i])


for i in range(len(times)):
  x = np.asarray(df_NIR.index)
  y = np.asarray(df_NIR[times[i]].rolling(3).mean())/norm
  plt.plot(x, y, label=str(d1[i])+' ps', color=viridis(np.log(np.int(d1[i])+1)/np.log(6000)))

plt.plot(np.array(df_NIR.index), np.zeros(len(df_NIR.index), dtype=float, order='C'), color='black')

norm = 1

for name in sorted(Lit_data):

  if ('Clafton' in name) and ('P3HT-PCBM' in name):
    data = Lit_data[name]
    x = np.asarray(data.index)
    y = np.asarray(data['ΔA'])
    for elem in name.split():
      if 'ps' in elem:
        t=elem.split(sep='ps')[0]
    if int(t) == 1:
      norm = abs(y).max()
    plt.plot(x, y/norm, label=t + 'ps (Clafton et al. 2013)', linestyle='--', color=viridis(np.log(np.int(t)+1)/np.log(6000)))



plt.title("P3HT-PCBM NP Solution NIR Transient Spectroscopy")
plt.xlabel('Wavelength (nm)')
plt.ylabel('ΔA (normalized)')

plt.grid('both', 'both')
plt.legend(bbox_to_anchor=(1.2,0.5))
plt.show()

In [ ]:
#Kinetic algorithm
WLs_of_interest = [450, 590, 605, 700, 850, 930, 1000, 1200]
WL_UV = sorted(i for i in WLs_of_interest if i <= 800)
WL_NIR = sorted(i for i in WLs_of_interest if i > 800)

def find_nearest_dex(array, number, direction=None): 
    idx = -1
    if direction is None:
        ser = np.abs(array-number)
        idx = ser.get_loc(ser.min())
    elif direction == 'backward':
        _delta = number - array
        _delta_positive = _delta[_delta > 0]
        if not _delta_positive.empty:
            idx = _delta.get_loc((_delta_positive.min()))
    elif direction == 'forward':
        _delta = array - number
        _delta_positive = _delta[_delta >= 0]
        if not _delta_positive.empty:
            idx = _delta.get_loc(_delta_positive.min())
    return idx
colormap = cm.get_cmap('plasma')
styles = [':', '--', '-.', (0, (3, 5, 1, 5)), (0, (1, 10)), (0, (1, 1)), (0, (3, 10, 1, 10, 1, 10)), (0, (5, 1)), (0, (3, 10, 1, 10)), (0, (5, 5)),(0, (3, 1, 1, 1)),  (0, (1, 1))]
def k_chart(data, name, guess):
  dex = find_nearest_dex(data.index, guess)
  #dd = data.iloc[dex-10:dex+10].mean()
  #data2 = dd[dd.index>1]
  #data3 = dd[(dd.index>1) & (dd.index<10)]
  #data4 = dd[(dd.index>400) & (dd.index<4500)]
  #update = pd.DataFrame(data4, columns = ['vals'])
  #slope = linregress(np.asarray(data4.index), np.asarray(np.log10(abs(data4))))[0]
  #intercept = linregress(np.asarray(data4.index), np.asarray(np.log10(abs(data4))))[1]
  #stderr = linregress(np.asarray(data4.index), np.asarray(np.log10(abs(data4))))[4]
  #update['estimate'] = 10**(slope*update.index+intercept)
  #+" -- Slope: "+str(np.around(slope, 6))+ " Std Err: "+str(np.around(stderr,8))

  
  
  i = WLs.index(guess)
  label = str(guess)+' nm'

  peak_to_norm =  data.iloc[dex-10:dex+10].mean().max()
  trough_to_norm = data.iloc[dex-10:dex+10].mean().min()
  norm = max(abs(peak_to_norm), abs(trough_to_norm))
  x = np.asarray(data.columns)
  y = np.asarray(data.iloc[dex-10:dex+10].mean())/norm
  plt.plot(x,y, label=label, linewidth=2, color=colormap(i/len(WLs)), linestyle=styles[i])
  #plt.plot(np.asarray(update.index), np.asarray(update['estimate']), linewidth=0.8, color='black')
  plt.ylabel('ΔA (normalized)')
  plt.yscale('linear')
  plt.xscale('log')
  plt.xlim(0.01,6900)
  plt.xlabel('Time (ps)')
def k_chart_bk(data, guess, label):
  f,(ax,ax2) = plt.subplots(1,2,sharey=True, facecolor='w')

  # plot the same data on both axes
  ax.plot(x, y)
  ax2.plot(x, y)

  ax.set_xlim(0,7.5)
  ax2.set_xlim(40,42.5)

  # hide the spines between ax and ax2
  ax.spines['right'].set_visible(False)
  ax2.spines['left'].set_visible(False)
  ax.yaxis.tick_left()
  ax.tick_params(labelright='off')
  ax2.yaxis.tick_right()

# This looks pretty good, and was fairly painless, but you can get that
# cut-out diagonal lines look with just a bit more work. The important
# thing to know here is that in axes coordinates, which are always
# between 0-1, spine endpoints are at these locations (0,0), (0,1),
# (1,0), and (1,1).  Thus, we just need to put the diagonals in the
# appropriate corners of each of our axes, and so long as we use the
# right transform and disable clipping.

  d = .015 # how big to make the diagonal lines in axes coordinates
# arguments to pass plot, just so we don't keep repeating them
  kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
  ax.plot((1-d,1+d), (-d,+d), **kwargs)
  ax.plot((1-d,1+d),(1-d,1+d), **kwargs)

  kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
  ax2.plot((-d,+d), (1-d,1+d), **kwargs)
  ax2.plot((-d,+d), (-d,+d), **kwargs)

# What's cool about this is that now if we vary the distance between
# ax and ax2 via f.subplots_adjust(hspace=...) or plt.subplot_tool(),
# the diagonal lines will move accordingly, and stay right at the tips
# of the spines they are 'breaking'


for name in Spec_data:
  if 'UV-Vis' in name:
    WLs = WL_UV
  elif 'NIR' in name:
    WLs = WL_NIR
  for value in WLs:
    k_chart(Spec_data[name], name, value)
  title  = switch(name) + ': ' +str(np.around(fluence_translate(name), 1))+' uJ/pulse/cm^2'
  plt.title(title)
  plt.legend()
  plt.show()


**Note to self on Kinetics: Careful!** *Maybe use the raw data rather than the interpolated data. You don't want to change the curve via interpolation.*

In [ ]:
#Kinetics 2: Compare same WL at different fluences
#Choose how to group
cond1 = ['UV-Vis', 'NIR']
cond2 = ['NPs', 'bulk']
cond3 = ['film', 'solution']
def group(full_dataset, *args):
  groups ={}
  #create all group types
  np.product(len(cond))
  
  for name in full_dataset:
  if all(x in name for x in mesh):
    if name not in groups:
      groups.update({mesh:name})
    elif name in groups:
      groups[mesh].append(name)  
          print(vi)
groups={}
for name in Spec_data:

print(groups)        
          
    



#Kinetics 3: Create a function which subtracts normalized kinetics at one WL from another and plot it (e.g. remove P3HT exciton)
def averaged_kinetics_norm(data, guess):
  dex = find_nearest_dex(data.index, guess)
  peak_to_norm =  data.iloc[dex-10:dex+10].mean().max()
  trough_to_norm = data.iloc[dex-10:dex+10].mean().min()
  norm = max(abs(peak_to_norm), abs(trough_to_norm))
  if abs(trough_to_norm) > abs(peak_to_norm):
    norm = -1*norm
  x = np.asarray(data.columns)
  y = np.asarray(data.iloc[dex-10:dex+10].mean())/norm
  return x, y

def isolate_polaron(data, exc_wl, pol_wl):
  x_exc, y_exc = averaged_kinetics_norm(data, exc_wl)
  x_pol, y_pol = averaged_kinetics_norm(data, pol_wl)
  f = interp1d(x_pol, y_pol)
  y_pol_approx = f(x_exc)
  pol_sig = y_pol_approx - y_exc
  pol_sig_norm = pol_sig/np.nanmax(pol_sig)
  plt.plot(x_exc, y_exc, label = 'Exciton signature: ' + str(exc_wl), linestyle='-')
  plt.plot(x_exc, pol_sig_norm, label='Inferred polaron: ' +str(pol_wl), linestyle=':')
  plt.plot(x_pol, y_pol, label='Raw signature: '+str(pol_wl), linestyle='-.')
  plt.xscale('log')
  plt.xlim(0.01,6500)
  plt.legend()
  plt.show()

for name in Spec_data:
  if 'NIR' in name:
    data = Spec_data[name]
    isolate_polaron(data, 1200, 1000)


#Kinetics 4: Fitting models!